In [9]:
####################################Imports and Initial Feed Rip###################################################



#Requires PySpark, installation information can be found here: https://www.dataquest.io/blog/installing-pyspark/
#import PySpark as ps 
import feedparser as fp, pandas as pd, datetime, time, string
from bs4 import BeautifulSoup 
from functools import reduce

#Get the News Feed and Slice it to only include the news items
feed = fp.parse('http://feeds.huffingtonpost.com/c/35496/f/677097/index.rss')
feed_items = feed['items']

###################################################################################################################

In [6]:
#################################Create the Cleaned News Items DataFrame###########################################

#Import the raw data into a DataFrame
raw_news_dataframe = pd.DataFrame(feed_items)

#Clean the DataFrame
def dataframe_cleaner(dataframe): 
    being_cleaned_df = pd.DataFrame(raw_news_dataframe[['author','id','title','summary']])
    
    for i in range(0,len(being_cleaned_df['summary'])):
        soup = BeautifulSoup(being_cleaned_df['summary'][i],'lxml')
        being_cleaned_df['summary'][i] = soup.get_text()
    
    return being_cleaned_df
    
#Create the cleaned Dataframe    
news_df = dataframe_cleaner(raw_news_dataframe)
news_df.head(1)

###################################################################################################################

,author,id,title,summary
0,Zeba Blay,http://www.huffingtonpost.com/2016/03/24/meet-...,Meet The Social Media Star Changing The Way We...,"If you don't know who Yagazie Emezi is, it's t..."


In [7]:
#Load in the Sentiment DataFrame
sentiment_df = pd.read_csv('sentiment_DataFrame.csv')
sentiment_df = sentiment_df.ix[:,'type':]
sentiment_df.head(1)

,type,word,position,stemmed,polarity
0,0.5,abandoned,adj,n,-1


In [19]:
##########################Perform Sentiment Analysis on the Title and Body of the Articles#########################
def title_analysis(title):
    tokenized_title = dict.fromkeys(title.split(' '))                          
                                
    for key in tokenized_title:
        tokenized_title[key] = [sentiment_df['type'][i] * sentiment_df['polarity'][i]
                               for i in range(0,len(sentiment_df.index)) 
                               if key.lower() == sentiment_df['word'][i]]
    
    total_sentiment = sum([item for sublist in list(tokenized_title.values()) for item in sublist])
    normalized_sentiment = total_sentiment / len(tokenized_title)
    
    return normalized_sentiment

def summary_analysis(summary):
    tokenized_summary = dict.fromkeys(summary.split(' '))
    
    for key in tokenized_summary:
        tokenized_summary[key] = [sentiment_df['type'][i] * sentiment_df['polarity'][i]
                               for i in range(0,len(sentiment_df.index)) 
                               if key.lower() == sentiment_df['word'][i]]
    
    total_sentiment = sum([item for sublist in list(tokenized_summary.values()) for item in sublist])
    normalized_sentiment = total_sentiment / len(tokenized_summary)
    return normalized_sentiment

def sentiment_analysis(dataframe):
    analyzing_df = dataframe
    analyzing_df['normalized_title_sentiment'] = list(map(lambda x: title_analysis(x),analyzing_df['title']))
    analyzing_df['normalized_summary_sentiment'] = list(map(lambda x: summary_analysis(x),analyzing_df['summary']))
    return analyzing_df

start_time = time.time()
analyzed_articles_df = sentiment_analysis(news_df)
elapsed_time = time.time() - start_time
print('Elapsed Time: ' + str(elapsed_time))
analyzed_articles_df.head()

###################################################################################################################

Elapsed Time: 450.0163402557373


,author,id,title,summary,normalized_title_sentiment,normalized_summary_sentiment
0,Zeba Blay,http://www.huffingtonpost.com/2016/03/24/meet-...,Meet The Social Media Star Changing The Way We...,"If you don't know who Yagazie Emezi is, it's t...",0.2000,0.072957
1,Damon Beres,http://www.huffingtonpost.com/2016/03/30/micro...,Society Needs To Decide Where Tech Is Taking U...,Will we control technology? Or will technolog...,0.0000,0.051402
2,Casey Williams,http://www.huffingtonpost.com/2016/03/30/micro...,"Microsoft's Chat Bot Re-Activates, Immediately...",March 30 (Reuters) - Almost a week after be...,-0.1250,0.008065
3,Damon Beres,http://www.huffingtonpost.com/2016/03/30/gta-v...,Thousands Are Watching A Virtual Deer Run Amok...,Watch live video from bwatanabe on www.twitch...,0.0000,0.024038
4,Daniel Burrus,http://www.huffingtonpost.com/daniel-burrus/te...,Technology Is Disrupting Crime -- Are You Ready?,Crime rates have been recorded for as long a...,-0.0625,-0.024752
